# Question Answering in Documenti

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
loader = TextLoader('data/dialoghi/dialogo.txt')
documents = loader.load()

In [3]:
for e in documents:
    print(e)

page_content='Al negozio di abbigliamento
Commesso - Buongiorno, posso aiutarla?
Cliente - Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?
Commesso - Certo, che taglia?
Cliente - Porto una 50
Commesso - Eccoli qua!
Cliente - Dove sono i camerini?
Commesso - I camerini sono là in fondo a destra, accanto alle scale
Cliente - Perfetto, grazie

Commesso - Come vanno?
Cliente - Il modello mi piace, ma sono un po' stretti. Posso provare una taglia più larga?
Commesso - Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore, vuole provare lo stesso modello in marrone?
Cliente - No, grazie, il marrone proprio non mi piace. Non avete altri colori in questa taglia?
Commesso - Allora, nella taglia 52 abbiamo il marrone, il rosso e il grigio.
Cliente - Vabbè, li provo in grigio, vediamo come mi stanno.

Cliente - Ho provato i pantaloni, anche in grigio sono proprio belli e la taglia è perfetta! Quanto costano?
Commesso - Costano 85€
Cliente - Ma non sono in sconto

In [4]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=os.getenv("openai_key"))

In [6]:
# Gli splitter sono degli oggetti messi a disposizione da LangChain per suddividere un testo in chunk più piccoli
# In questo modo, testi lunghi possono essere gestiti più facilmente con le informazioni presenti splittate in più chunk
# oppure per essere processati da un LLM che ha un limite sul numero di token in input
#
# In questo esempio utilizziamo il RecursiveCharacterTextSplitter che cerca di splittare il testo in maniera "smart"
# creando chunk con una dimensione massima specificata (chunk_size) ma cercando di non spezzare frasi a metà andando 
# a chunkizzare il testo sulla punteggiatura (non la prima punteggiatura che trova, ma la più vicina alla fine del chunk)
#
# Inoltre per evitare di perdere troppo il contesto tra un chunk e l'altro, è possibile specificare un overlap (chunk_overlap)
# che indica quanti caratteri devono essere ripetuti tra un chunk e l'altro
#
# Non ci sono regole fisse sulla dimesione del chunk e sull'overlap e dipende da ogni caso specifico, 
# ma certamente non bisogna andare oltre il limite di token del modello di embedding che andiamo ad utilizzare
#
# https://python.langchain.com/docs/how_to/recursive_text_splitter/
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)

splits = text_splitter.split_documents(documents)


# Alcuni altri splitter standard:
# * CharacterTextSplitter - split fissi su un set di caratteri
# * NLTKTextSplitter - usa la libreria NLTK per identificare singole frasi da usare come chunk
# * SpacyTextSplitter - usa la libreria Spacy per identificare singole frasi da usare come chunk
# * MarkdownTextSplitter - sfrutta la sintassi Markdown per identificare sezioni atomiche di testo da usare come chunk (intestazioni, liste, blocchi di codice, ...)
# * LatexTextSplitter - sfrutta la sintassi Latex per identificare sezioni atomiche di testo da usare come chunk (sezioni, sottosezioni, equazioni)b

In [7]:
splits

[Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Al negozio di abbigliamento\nCommesso - Buongiorno, posso aiutarla?\nCliente - Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?\nCommesso - Certo, che taglia?'),
 Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Commesso - Certo, che taglia?\nCliente - Porto una 50\nCommesso - Eccoli qua!\nCliente - Dove sono i camerini?\nCommesso - I camerini sono là in fondo a destra, accanto alle scale'),
 Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Cliente - Perfetto, grazie'),
 Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content="Commesso - Come vanno?\nCliente - Il modello mi piace, ma sono un po' stretti. Posso provare una taglia più larga?"),
 Document(metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Commesso - Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore, vuole provare lo stesso modello in ma

In [8]:
for split in splits:
    print(split.page_content)
    print("-----")

Al negozio di abbigliamento
Commesso - Buongiorno, posso aiutarla?
Cliente - Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?
Commesso - Certo, che taglia?
-----
Commesso - Certo, che taglia?
Cliente - Porto una 50
Commesso - Eccoli qua!
Cliente - Dove sono i camerini?
Commesso - I camerini sono là in fondo a destra, accanto alle scale
-----
Cliente - Perfetto, grazie
-----
Commesso - Come vanno?
Cliente - Il modello mi piace, ma sono un po' stretti. Posso provare una taglia più larga?
-----
Commesso - Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore, vuole provare lo stesso modello in marrone?
-----
Cliente - No, grazie, il marrone proprio non mi piace. Non avete altri colori in questa taglia?
Commesso - Allora, nella taglia 52 abbiamo il marrone, il rosso e il grigio.
-----
Cliente - Vabbè, li provo in grigio, vediamo come mi stanno.
-----
Cliente - Ho provato i pantaloni, anche in grigio sono proprio belli e la taglia è perfetta! Quanto costano

In [9]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings()) 

Un retriever è un componente di LangChain che ha lo scopo di recuperare informazioni da una base di conoscenza vettoriale. Di base un retriever è un componente di LangChain che riceve in input una query e ci restituisce in outuput la lista di documenti più "vicina". Un retriever per funzionare, ovviamente, va "agganciato" ad un vectorstore.

Il vectorstore che abbiamo appena creato si basa su ChromaDB ma è comunque un componente di LangChain e, per questo, ci mette a disposizione un metodo as_retriever() che ci restituisce un retriever già "agganciato" al vectore store in questione.

Questo meccanismo è uno dei tanti meccanismi modulari di LangChain, infatti se modificassimo la tecnologia sotto il vector store utilizzato, il resto del codice resterebbe identico, perché ogni vector store creato utilizzando i metodi messi a disposizione da LangChain, ci metteranno sempre a disposizione il metodo as_retriever() 

https://python.langchain.com/docs/concepts/retrievers/

In [11]:
retriever = vectorstore.as_retriever()

Il task di RAG è un task piuttosto comune e per questo motivo esistono dei prompt standard che possiamo utilizzare
LangSmith è un hub di prompt standard messi a disposizione dalla comunità di LangChain e possiamo recuperarli facilmente
(https://smith.langchain.com/hub)

Proprio perché questi prompt standard sono stati altamente utilizzati, sono presenti nella maggiorparte degli esempi usati per il fine-tuning di questi modelli  per i compiti di RAG e questo fa si che, utilizzare questi prompt standard, migliori generalmente le performance

In [13]:
prompt = hub.pull("rlm/rag-prompt")

print(prompt.input_variables)
print()
print(prompt.messages[0].prompt.template)

['context', 'question']

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


/home/simone/aaamc/venv/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
retriever.invoke("Dove sono i camerini")

[Document(id='db46108c-4e4a-4fc8-98b8-c64c7cef3956', metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Commesso - Certo, che taglia?\nCliente - Porto una 50\nCommesso - Eccoli qua!\nCliente - Dove sono i camerini?\nCommesso - I camerini sono là in fondo a destra, accanto alle scale'),
 Document(id='156c2c06-a7ef-4b67-8ac7-001e81b283c9', metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Cliente - Vabbè, li provo in grigio, vediamo come mi stanno.'),
 Document(id='702b17ca-bf50-4ac7-8d2e-3c79129a8b3f', metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Al negozio di abbigliamento\nCommesso - Buongiorno, posso aiutarla?\nCliente - Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?\nCommesso - Certo, che taglia?'),
 Document(id='9415f55e-14b7-4208-828e-cff4f17f1bbf', metadata={'source': 'data/dialoghi/dialogo.txt'}, page_content='Commesso - Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore, vuole provare lo stesso

In [15]:
# Metodo molto semplice per formattare i documenti recuperati dal retriever
# in un unico testo da passare poi al prompt
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


(retriever | format_docs).invoke("Dove sono i camerini")

'Commesso - Certo, che taglia?\nCliente - Porto una 50\nCommesso - Eccoli qua!\nCliente - Dove sono i camerini?\nCommesso - I camerini sono là in fondo a destra, accanto alle scale\n\nCliente - Vabbè, li provo in grigio, vediamo come mi stanno.\n\nAl negozio di abbigliamento\nCommesso - Buongiorno, posso aiutarla?\nCliente - Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?\nCommesso - Certo, che taglia?\n\nCommesso - Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore, vuole provare lo stesso modello in marrone?'

In [26]:
from typing import Dict, Any
from langchain.schema.runnable import RunnableLambda

def debug_print(data: Dict[str, Any]) -> Dict[str, Any]:
    print(data.messages[0].content)
    return data

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | RunnableLambda(debug_print)
    | llm
    | StrOutputParser()
)


rag_chain.invoke("dove sono i camerini?")

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: dove sono i camerini? 
Context: Commesso - Certo, che taglia?
Cliente - Porto una 50
Commesso - Eccoli qua!
Cliente - Dove sono i camerini?
Commesso - I camerini sono là in fondo a destra, accanto alle scale

Al negozio di abbigliamento
Commesso - Buongiorno, posso aiutarla?
Cliente - Sì, grazie. Ho visto un paio di pantaloni neri in vetrina, posso provarli?
Commesso - Certo, che taglia?

Commesso - Oh, mi dispiace, abbiamo finito la taglia 52 in questo colore, vuole provare lo stesso modello in marrone?

Cliente - No, grazie, il marrone proprio non mi piace. Non avete altri colori in questa taglia?
Commesso - Allora, nella taglia 52 abbiamo il marrone, il rosso e il grigio. 
Answer:


'I camerini sono in fondo a destra, accanto alle scale.'

In [27]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
rag_chain.invoke("qual'è la taglia del cliente?")

'La taglia del cliente è 52.'

In [29]:
rag_chain.invoke("in che negozio è ambientata la scena?")

'La scena è ambientata in un negozio di abbigliamento.'

In [30]:
rag_chain.invoke("dove sono i camerini?")

'I camerini sono in fondo a destra, accanto alle scale.'